# Case study II - HTTP server

## Java compilation unit

A Java program is normally a collection of definitions of classes (and interfaces). At least one of the classes (main class) must be declared ```public``` and must contain a definition of method named ```main```. This method must be declared ```public```, ```static``` and ```void```. The definition of the ```main``` method must declare exactly one parameter of type ```String[]``` which represents the array of the parameters passed to the program from the command line, at the time of its execution.

In [ ]:
class A
{

}

In [ ]:
class B
{

}

In [ ]:
public class C
{
    public static void main(String[] args)
    {
    
    }
}

The classes must be stored in one or more files (compilation unit), but any public class must be stored in a file named after this class, with extension ```.java```. In the above case, the public class ```C``` must be stored in a file ```C.java``` (note the case sensitivity). Other (not public) classes may be stored together with one of the public classes (or in separate files).

In order to compile the program, you should invoke java compiler (```javac```) and pass to it the appropriate name of file.

```$ javac C.java```

The compiler then produces the so called "bytecodes" - executables for the Java interpreter (virtual machine). The bytecodes of each class are stored in separate files, named after the class name, but with extension ```.class```.

In order to run the program, you should invoke the interpreter (```java```) and pass to it the name of the main class (not the name of file), which initiates the execution of the program.

```$ java C```

Java programs are usually distributer as "jar files". These are compressed archives containing bytecode (```.class```)files, data files and a manifest file. The manifest describes the Java archive (jar file) and is stored in a file named ```MANIFEST.MF```, located under the META-INF directory in the jar archive. It contains a list of key and value pairs, called headers or attributes, grouped into sections. These headers supply metadata that provide information on the jar archive, such as the versions of packages or what application class to execute (Main-Class header). Jar archives can be created using ```jar``` tool.

```$ jar cf file.jar classes/```

Documentation of the classes that make the project can be created using ```javadoc``` tool. Popular Integrated Development Environments (IDEs), such as Netbeans, Eclipse or IntelliJ, allow to simplify and automate use of the Java compiler, interpreter, jar tool and other tools delivered with the Java platform.

## Case foundation

The case involves creating simple implementation of the HTTP server by developing further the starting version you can find below. This approach allows to apply many concepts of network programming in Java language - use of classes and objects, multithreading, use of the Java API (sockets and streams) and use of the data structures (ArrayList). You will need the web browser as a client for your HTTP server.

The starting version implements the server which listens on the port 80 for the incoming connections. When the web browser successfully connects, the server creates the appropriate streams which will be used to read the HTTP request and send the HTTP response back to the web browser - a text stream to for reading HTTP request data (text) and a binary stream to send HTTP response, as it may contain text/html or binary data (such as graphics) depending of its mime type. Then, the starting version of the server processes the HTTP request and sends some fixed HTTP response back to the browser. Exercises are aimed at implementing the process of retrieving information from the HTTP request and use them to open, read and send to the broser the content of the requested web page (containg text and grahics). Further exercises are aimed to create multithreaded version of this http server and, finally, to create the HTTPS server.

## Exercise 1

Create the appropriate compilation unit (file) containg the classes below (you may, or you may not define a package). Then compile it with the Java compiler and run with the Java interpeter. Analyze the code and assign its actions and functionalities to its structure. If it is possible, discuss this with your peer.

```java
import java.io.*;
import java.net.*;
 
public class ServerHTTP
{
   public static void main(String[] args) throws IOException                
   {                                                                       
      ServerSocket serv=new ServerSocket(80);                              
 
      while(true)                                                          
      {                                                                    
         //accepting connection                                            
         System.out.println("Waiting for connection...");               
         Socket sock=serv.accept();                                        
 
         //data streams                                               
         InputStream is=sock.getInputStream();                             
         OutputStream os=sock.getOutputStream();                           
         BufferedReader inp=new BufferedReader(new InputStreamReader(is)); 
         DataOutputStream outp=new DataOutputStream(os);                   
 
         //reading request                                     
         String request=inp.readLine();                                    
 
         //sending response                                  
         if(request.startsWith("GET"))                                     
         {                                                                 
            //response header                                              
            outp.writeBytes("HTTP/1.0 200 OK\r\n");                        
            outp.writeBytes("Content-Type: text/html\r\n");                
            outp.writeBytes("Content-Length: \r\n");                       
            outp.writeBytes("\r\n");                                       
 
            //response body                                                
            outp.writeBytes("<html>\r\n");                                 
            outp.writeBytes("<H1>Strona testowa</H1>\r\n");                
            outp.writeBytes("</html>\r\n");                                
         }                                                                 
         else                                                              
         {                                                                 
            outp.writeBytes("HTTP/1.1 501 Not supported.\r\n");            
         }                                                                 
 
         //closing streams                                             
         inp.close();                                                      
         outp.close();                                                     
         sock.close();                                                     
      }                                                                    
   }                                                                       
}
```

## Exercise 2

Implement the retrieving data from HTTP request received by the server from the web browser. You will need the URL of the resource demanded by the web browser. You may retrieve these data from the first line of the HTTP request. Below you can find an example of it.

In [3]:
String request = "GET /index.html HTTP/1.1";
System.out.println(request);

GET /index.html HTTP/1.1


The request line (first line of a request) contains three tokens separated by spaces - "GET", the URL of the resource and "HTTP/1.1". In order to retrieve the URL, you may use the methods of the Java API String class.

In [5]:
String[] tab = request.split(" ");

for(int i=0;i<3;i++)
{
    System.out.println(tab[i]);
}

GET
/index.html
HTTP/1.1


Now you are able to retrieve the name of file requested by the web browser.

In [10]:
String fileName = tab[1].substring(1);
System.out.println(fileName);

index.html


Now, you are ready to open the file of specified name, read its contents to the byte buffer and, finally, write the contents of the buffer to the appropriate output stream related with the HTTP request being sent back to the web browser. Note, that unlike the HTTP request (which contains text data), HTTP response may contain either text (html) or binary (graphics), depending on its mime type. As a result, you need to use binary stream (DataOutputStream) for the response, while it is sufficient to use text stream (BufferedReader) for the request. Below you find an example of how to accomplished this task.

```java
FileInputStream fis = new FileInputStream(fileName);
 
byte[] bufor;
bufor=new byte[1024];
int n=0;
 
while ((n = fis.read(bufor)) != -1 )
{
	outp.write(buffer, 0, n);
}
```

## Exercise 3

Using the starting version below, develop further your HTTP server to create its multithreaded version.

```java
import java.io.*;
import java.net.*;
 
class HandlingRequest extends Thread
{
   Socket sock;
 
   HandlingRequest(Socket klientSocket)  
   {                                   
      this.sock=klientSocket;          
   }                                   
 
   public void run() 
   {                 
 
   }                 
}
 
public class ServerHTTP
{
   public static void main(String[] args) throws IOException     
   {                                                            
      ServerSocket serv=new ServerSocket(80);                   
 
      while(true)                                               
      {                                                         
         //accepting connection                                 
         System.out.println("Oczekiwanie na polaczenie...");    
         Socket sock=serv.accept();                             
 
         //creating and starting handling request thread             
         new HandlingRequest(sock).start();                      
      }                                                         
   }                                                            
}
```

## Exercise 4

Using the starting version below, develop further your HTTP server to create its HTTPS version.

```java
import java.io.*;
import java.net.*;
import javax.net.ssl.*;
 
public class ServerHTTPS
{
 
   public static void main(String[] args) throws IOException                     
   {                                                                             
      SSLServerSocketFactory fact;                                               
      fact = (SSLServerSocketFactory)SSLServerSocketFactory.getDefault();        
      ServerSocket servsock = fact.createServerSocket(8080);                     
 
      while (true)                                                               
      {                                                                          
         try                                                                     
         {                                                                       
            Socket sock = servsock.accept();                                     
 
            OutputStream out;                                                    
            out=sock.getOutputStream();                                          
 
            BufferedReader in;                                                   
            in=new BufferedReader(new InputStreamReader(sock.getInputStream())); 
 
 
 
            in.close();                                                          
            out.close();                                                         
            sock.close();                                                        
         }                                                                       
         catch (Exception e)                                                     
         {                                                                       
            e.printStackTrace();                                                 
         }                                                                       
      }                                                                          
   }                                                                             
}
```